The KNN algorithm is very simple to implement, as it does not need to be trained. The training phase merely stores the training data. For each test point, we calculate the distance of that data point to every existing data point and find the $K$ closest ones. What we return is the the most common amongst the top k classification nearest to the test point. Here's the pseudocode for _K_ Nearest Neighbors:

```
kNN:

    Learn:
        Store training set T to X_train: X_train <-- T


    Predict:
        for every point xp in X_predict:
            for every point x in X_train:
                calculate the distance d in D between x and xp
            sort D in increasing order
            take the "k" items in X_train with the smallest distances to x
            return the majority class among these k items
```

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and evey other point in the data!


# Distance Metrics - Note

As explained above, KNN assigns a class to the test point based on the majority class of  KK  nearest neighbours. In general, euclidean distance is used to find nearest neighbours, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbours. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is Mahalanobis distance. Here's the wikipedia page on Mahalanobis distance. https://en.wikipedia.org/wiki/Mahalanobis_distance Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit learn scaling package. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [138]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report


In [139]:
# 1. load the data      0 = iris-setosa, 1 = iris-virginica, 2 = iris-versicolor
iris = pd.read_csv('iris.csv')  # load data
iris.head()
iris = iris.replace('Iris-setosa',0)       # create dummy variables
iris = iris.replace('Iris-virginica',1)
iris = iris.replace('Iris-versicolor',2)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [140]:
X = iris.iloc[:,0:4]
y = iris.iloc[:,4]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5, random_state=25)
X_test.shape

(75, 4)

In [185]:
distance = pd.DataFrame(np.zeros((75,4)))
def euclidianDistance(a, b):
    for i in range(len(a)):
        for j in range(4):
            distance.iloc[i][j] += np.square(a.iloc[i,j] - b.iloc[i,j])
            distance2 = np.sqrt(distance)
    
    return distance2


In [208]:
dist = pd.DataFrame(np.zeros((75)))

In [211]:

def euclidianDistance(a, b,length):
    for i in range(length):
        dist.iloc[i] += np.square(a[i] - b[i])
        
    
    return np.sqrt(dist)

In [212]:
euclidianDistance(iris.sepal_length,iris.sepal_width,75)

,0
0,2.262742
1,2.483948
2,2.193171
3,2.193171
4,2.126029
5,2.193171
6,2.000000
7,2.262742
8,2.193171
9,2.408319


In [187]:
# 2. define KNN algorithm

def KNN(data, query, k, distance, choice):
    neighbor_distances_and_indices = []
    
    # 3. For each example in the data
    for index, example in enumerate(data):
        distance = distance_fn(example[:-1], query)  # distance calculation
        
        # 3.2 Add the distance and the index of the example to an ordered collection
        neighbor_distances_and_indices.append((distance, index))
    
    # 4. Sort the ordered collection of distances and indices from
    # smallest to largest (in ascending order) by the distances
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # 5. Pick the first K entries from the sorted collection
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # 6. Get the labels of the selected K entries
    k_nearest_labels = [data[i][1] for distance, i in k_nearest_distances_and_indices]

    # 7. If regression (choice_fn = mean), return the average of the K labels
    # 8. If classification (choice_fn = mode), return the mode of the K labels
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)
    